In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import itertools
import holidays

# Loading data into dataframe -- energy prices for houston, north, south, and west load zones, predicted and actual energy load for all four load zones, and oil prices

In [31]:
df = pd.read_csv(r"C:\Users\achra\OneDrive\Documents\GitHub\ERCOT_Price_Prediction_Stat_413_Final_Project\Data\CombinedData2023.csv")

In [34]:
df['Oil Price'] = df['Oil Price'].ffill()
df

,Date,Time,North Price,West Price,South Price,Houston Price,North Load,West Load,South Load,Houston Load,North Predicted Load,West Predicted Load,South Predicted Load,Houston Predicted Load,Oil Price
0,01/01/2023,00:00,-2.1200,-2.0200,-2.1100,-2.1150,11118.248963,7102.853761,8891.418391,8578.637876,14210.616112,7615.188874,10261.255994,11152.307651,76.87
1,01/01/2023,01:00,-1.1500,-0.8950,-1.1250,-1.1375,11115.263857,7107.853761,8871.418391,8514.637876,14297.425469,7661.026271,10166.597073,11011.450533,76.87
2,01/01/2023,02:00,-1.1700,-1.1275,-1.1675,-1.1700,10855.755296,7031.047191,8779.863470,8345.632899,14375.662482,7669.107039,10155.831781,10867.511020,76.87
3,01/01/2023,03:00,-0.0175,0.0275,-0.0125,-0.0150,10641.477018,7012.444478,8589.953921,8125.706879,14610.251706,7720.992165,10144.531996,10957.793669,76.87
4,01/01/2023,04:00,-0.8125,-0.8125,-0.8125,-0.8125,10574.197398,7035.195182,8396.254102,7995.083655,14931.854721,7762.281965,10258.219673,11067.944014,76.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,12/31/2023,19:00,14.2950,15.0000,14.3675,14.3325,14151.533311,8620.619728,10715.416266,12189.507916,15460.690742,8021.642107,11972.059826,13026.298976,71.89
8757,12/31/2023,20:00,12.6250,13.1925,11.9300,12.5375,13922.923678,8534.075954,10376.214504,11918.024426,15184.016847,8017.744551,11704.179439,12768.119151,71.89
8758,12/31/2023,21:00,11.6700,13.0250,2.5550,10.2650,13726.395295,8542.625513,9989.361479,11634.439863,14843.335871,7935.433637,11288.477649,12528.756783,71.89
8759,12/31/2023,22:00,11.2900,12.6325,2.5775,9.8925,13508.403724,8612.699674,9632.467050,11360.099483,14453.674783,7868.521980,10984.821975,12163.892862,71.89


# Feature Preprocessing before training LSTM Model:

    1) Need to engineer lags for all energy price columns, and all actual load columns. Lags will be 1 hour, 24 hours, and 168 hours for price columns, and  24 hours for load columns.
    2) Need to create exponential moving average (EMA) mean price and EMA price standard deviation variables for north price, west price, south price, and houston price. 
    3) Need to generate pairwise differences between north, west, south, and houston prices, but using the lagged version (168 hours ago) as the actual variable
    4) Remove first week of data due to lag features being null.
    5) Delete predicted load columns, save the last week of predicted load values to replace actual load with for the test set.
    6) Encode month cyclically as sin and cos values.
    7) Encode day of the week cyclically as sin and cos values.
    8) Add binary holiday column to indicate whether a price observation was during a holiday or not.
    9) Use standard scaler to normalize all numerical features.

In [26]:
# Engineering Lagged Features

def create_lagged_features(df, price_cols_and_lags, load_cols_and_lags):
    """
    Create lagged features for price and load columns.
    price_cols_and_lags: dictionary with keys as column names and values as a list of lags
    load_cols_and_lags: dictionary with keys as column names and values as a list oflags
    """
    new_df = df.copy()
    for col, lags in price_cols_and_lags.items():
        for lag in lags:
            new_df[f'{col}_lag{lag}'] = new_df[col].shift(lag)
    
    for col, lags in load_cols_and_lags.items():
        for lag in lags:
            new_df[f'{col}_lag{lag}'] = new_df[col].shift(lag)
    
    return new_df

price_cols_and_lags = {col: [1,24,168] for col in ['North Price', 'Houston Price', 'South Price', 'West Price']}
load_cols_and_lags = {col: [24] for col in ['North Load', 'Houston Load', 'South Load', 'West Load']}


In [43]:
def create_ema_mean_and_sd(df, ema_cols, span = 168):
    """
        Create Exponential Moving Average (EMA) Mean and Exponential Moving Average Standard Deviation (SD) for specified columns.
        Parameters:
        df (pandas.DataFrame): The input DataFrame containing the data.
        ema_cols (list of str): List of column names for which to calculate the EMA mean and SD.
        span (int, optional): The span for the EMA calculation. Default is 168.
        Returns:
        pandas.DataFrame: A new DataFrame with the original columns and additional columns for EMA mean and SD for each specified column.
    """
    new_df = df.copy()
    for col in ema_cols:
        new_df[f'{col}_ema_mean'] = new_df[col].ewm(span= span).mean()
        new_df[f'{col}_ema_std'] = new_df[col].ewm(span= span).std()
    
    return new_df

ema_cols = ['North Price', 'Houston Price', 'South Price', 'West Price']

In [ ]:
def create_pairwise_differences_and_lags(df, price_cols, lag = 168):
    new_df = df.copy()
    pairwise_price_combinations = list(itertools.combinations(price_cols, 2))
    for price1, price2 in pairwise_price_combinations:
        new_df["{}_{}_diff".format(price1, price2)] = new_df[price1] - new_df[price2]
        new_df["{}_{}_diff_lag{}".format(price1, price2, lag)] = new_df["{}_{}_diff".format(price1, price2)].shift(lag)
        new_df.drop(labels = ["{}_{}_diff".format(price1, price2)], axis = 1, inplace = True)
    return new_df


   North Price  Houston Price  South Price  West Price  \
0           10             15           12          18   
1           20             25           22          28   
2           30             35           32          38   
3           40             45           42          48   
4           50             55           52          58   
5           60             65           62          68   
6           70             75           72          78   
7           80             85           82          88   
8           90             95           92          98   
9          100            105          102         108   

   North Price_Houston Price_diff_lag168  North Price_South Price_diff_lag168  \
0                                    NaN                                  NaN   
1                                    NaN                                  NaN   
2                                    NaN                                  NaN   
3                                    

In [48]:
def delete_rows_with_missing_values(df, rows = 168):
    """
    Delete rows with missing data due to adding lagged features.
    Parameters:
    df (pandas.DataFrame): The input DataFrame containing the data.
    rows (int, optional): The number of rows to delete. Default is 168.
    Returns:
    pandas.DataFrame: A new DataFrame with the specified number of rows deleted.
    """
    new_df = df.copy()
    new_df = new_df.iloc[rows:]
    return new_df

In [ ]:
def delete_predicted_load(df, load_cols, predicted_load_cols, test_data_size):
    """
    Delete the predicted load columns, but replace the last test_data_size rows from the actual load values with the corresponding predicted load values.
    """
    new_df = df.copy()
    for load_col, predicted_load_col in zip(load_cols, predicted_load_cols):
        new_df.iloc[-test_data_size:, new_df.columns.get_loc(load_col)] = new_df.iloc[-test_data_size:,new_df.columns.get_loc(predicted_load_col)]
    for predicted_load_col in predicted_load_cols:
        new_df.drop(predicted_load_col, axis = 1, inplace = True)
    return new_df



In [62]:
def encode_month(df):
    """
    Encode the month as a cyclic feature
    """
    new_df = df.copy()
    new_df['month'] = pd.to_datetime(new_df['Date']).dt.month
    new_df['month_sin'] = np.sin(2 * np.pi * new_df['month'] / 12)
    new_df['month_cos'] = np.cos(2 * np.pi * new_df['month'] / 12)
    new_df.drop(labels = ['month'], axis = 1, inplace = True)
    return new_df

In [63]:
def encode_day_of_week(df):
    """
    Encode the day of the week as a cyclic feature
    """
    new_df = df.copy()
    new_df['day_of_week'] = pd.to_datetime(new_df['Date']).dt.dayofweek
    new_df["day_of_week_sin"] = np.sin(2 * np.pi * new_df["day_of_week"] / 7)
    new_df["day_of_week_cos"] = np.cos(2 * np.pi * new_df["day_of_week"] / 7)
    new_df.drop(labels = ["day_of_week"], axis = 1, inplace = True)
    return new_df


In [82]:
def encode_holidays(df):
    """
    Encode holidays as a binary feature
    """
    new_df = df.copy()
    tx_holidays = holidays.US(state = 'TX')
    new_df["Is_Holiday"] = new_df["Date"].apply(lambda x : 1 if x in tx_holidays else 0)
    return new_df


In [85]:
load_cols = ['North Load', 'Houston Load', 'South Load', 'West Load']
predicted_load_cols = ['North Predicted Load', 'Houston Predicted Load', 'South Predicted Load', 'West Predicted Load']
df_1 = create_lagged_features(df, price_cols_and_lags, load_cols_and_lags)
df_2 = create_ema_mean_and_sd(df_1, ema_cols)
df_3 = create_pairwise_differences_and_lags(df_2, ['North Price', 'Houston Price', 'South Price', 'West Price'])
df_4 = delete_rows_with_missing_values(df_3)
df_5 = encode_month(df_4)
df_6 = encode_day_of_week(df_5)
df_7 = encode_holidays(df_6)
df_final = delete_predicted_load(df_7, load_cols, predicted_load_cols, 168)

In [89]:
columns_to_scale = [col for col in df_final if col not in ['Date', 'Time','month_sin', 'month_cos', 'day_of_week_sin', 'day_of_week_cos', 'Is_Holiday']]
def scale_data(df, columns_to_scale):
    """
    Scale the data using StandardScaler
    """
    new_df = df.copy()
    scaler = StandardScaler()
    new_df[columns_to_scale] = scaler.fit_transform(new_df[columns_to_scale])
    return (new_df, scaler)

In [ ]:
def split_and_scale_data(df, columns_to_scale, test_size=168):
    """
    Split the data into training and test sets, and scale the numerical features.
    
    Parameters:
    df (pandas.DataFrame): The input DataFrame containing the data.
    columns_to_scale (list): List of column names to be scaled.
    test_size (int, optional): The number of rows to be used as the test set. Default is 168.
    
    Returns:
    tuple: A tuple containing the scaled training DataFrame, scaled test DataFrame, and the scaler object.
    """
    df_train = df.iloc[:-test_size]
    df_test = df.iloc[-test_size:]
    
    df_train_scaled, scaler = scale_data(df_train, columns_to_scale)
    df_test_scaled = df_test.copy()
    df_test_scaled[columns_to_scale] = scaler.transform(df_test[columns_to_scale])
    
    return df_train_scaled, df_test_scaled, scaler

df_train_scaled, df_test_scaled, scaler = split_and_scale_data(df_final, columns_to_scale)